# Azure AI Foundry trace 
You can trace your application with Azure AI Foundry project library. This provides support for tracing with OpenTelemetry.

> ✨ ***Note*** <br>
> Please check the prerequisites and set up your environment before running the code below. First follow steps to create an AI Project if you don't have one already. To enable tracing, first ensure your project has an attached Application Insights resource. Go to the Tracing page of your project in Azure AI Foundry portal and follow instructions to create or attach Application Insights.


## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. An Azure Subscription.
 1. An Azure AI project, see Create a project in Azure AI Foundry portal.
 1. An AI model supporting the Azure AI model inference API deployed through Azure AI Foundry.
 1. Ensure your project has an attached Application Insights resource
 1. Open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

For a dependency installation, run the code below to install the packages required to run it. 

```bash
pip install -r requirements.txt
```

## Set up your environment
Git clone the repository to your local machine. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.

In [25]:


import os
from opentelemetry import trace
import json
from openai import AzureOpenAI

from azure.ai.inference.models import (
    ToolMessage,
    AssistantMessage,
    ChatCompletionsToolCall,
    ChatCompletionsToolDefinition,
    FunctionDefinition,
)

# Install opentelemetry with command "pip install opentelemetry-sdk".
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, CompletionsFinishReason
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
load_dotenv(override=True)


azure_model_inference_endpoint = os.getenv("AZURE_MODEL_INFERENCE_ENDPOINT")
aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
project_connection_string = os.getenv("PROJECT_CONNECTION_STRING")
application_insights_connection_string = os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")

# 1. Copy the connection string from your Application Insights resource
The connection string is unique and specifies where the Azure Monitor OpenTelemetry Distro sends the telemetry it collects.
To copy the connection string:
<br>
- Go to the Overview pane of your Application Insights resource.
- Find your connection string.
- Hover over the connection string and select the Copy to clipboard icon.

![Migrate from instrumentation keys to connection strings](migrate-from-instrumentation-keys-to-connection-strings.png)

# 2. Configure Azure Monitor with AIProject client

In [ ]:
import sys
#Use Azure Monitor for OpenTelemetry via project client
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=project_connection_string,
)

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()

# Enable additional instrumentations if openai and langchain 
# langchain - https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-local-sdk?tabs=python#enable-tracing-for-langchain
# which are not included by Azure Monitor out of the box
project_client.telemetry.enable()
# you can also use the application insights connection string directly
configure_azure_monitor(connection_string=application_insights_connection_string)

# 3. Set up OpenAIInstrumentor to test Azure Open AI tracing 
- https://pypi.org/project/opentelemetry-instrumentation-openai-v2/

In [27]:
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

# OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT - Optional. Set to `true` to trace the content of chat messages, which may contain personal data. False by default.
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"

# Instrument Azure Open AI API 
OpenAIInstrumentor().instrument()

In [28]:
try:
    aoai_client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version
    )
except (ValueError, TypeError) as e:
    print(e)

In [ ]:
question = "Tell me the history of OpenAI and its models."

user_message = f"""
Question: {question}
"""

# Simple API Call
response = aoai_client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {"role": "user", "content": user_message},
    ],
  
)
print(response.choices[0].message.content)

Microsoft Corporation, one of the largest and most influential technology companies in the world, has a rich history that spans several decades. Here’s an overview of its development:

### Foundation and Early Years (1975-1980)

- **1975**: Microsoft was founded by Bill Gates and Paul Allen in Albuquerque, New Mexico. Initially, the company focused on developing and selling a version of the BASIC programming language for the Altair 8800 microcomputer.

- **1976-1980**: Following their initial success, Microsoft relocated to Washington State. The company began expanding its software offerings, creating a variety of programming languages and tools for different computer systems.

### Rise to Prominence (1980-1990)

- **1980**: Microsoft entered into a partnership with IBM to create an operating system for their first personal computer (PC). Although they didn’t develop the OS themselves, they purchased QDOS (Quick and Dirty Operating System) from Seattle Computer Products, modified it, a

In [37]:
response = aoai_client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {
            "role": "user",
            "content": "How many inches are in a feet?",
        },
    ],
)

print(response.choices[0].message.content)

There are 12 inches in a foot.


## Trace function calling with Azure OpenAI SDK

In [38]:
from opentelemetry import trace
#The tracer.start_as_current_span decorator will trace the function call and enable adding additional attributes
#to the span in the function implementation. Note that this will trace the function parameters and their values.
tracer = trace.get_tracer(__name__)

@tracer.start_as_current_span("get_temperature")  # type: ignore
def get_temperature(city: str) -> str:

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_city", city)

    if city == "Seattle":
        return "75"
    elif city == "New York City":
        return "80"
    else:
        return "Unavailable"


# [END trace_function]


def get_weather(city: str) -> str:
    if city == "Seattle":
        return "Nice weather"
    elif city == "New York City":
        return "Good weather"
    else:
        return "Unavailable"




weather_description = ChatCompletionsToolDefinition(
    function=FunctionDefinition(
        name="get_weather",
        description="Returns description of the weather in the specified city",
        parameters={
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city for which weather info is requested",
                },
            },
            "required": ["city"],
        },
    )
)

temperature_in_city = ChatCompletionsToolDefinition(
    function=FunctionDefinition(
        name="get_temperature",
        description="Returns the current temperature for the specified city",
        parameters={
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city for which temperature info is requested",
                },
            },
            "required": ["city"],
        },
    )
)

In [39]:
messages = [
        SystemMessage("You are a helpful assistant."),
        UserMessage("What is the weather and temperature in Korea?"),
    ]

tools = [weather_description, temperature_in_city]

response = aoai_client.chat.completions.create(model=aoai_deployment_name, messages=messages, tools=tools)

if response.choices[0].finish_reason == "tool_calls":
    messages.append(AssistantMessage(
        tool_calls=response.choices[0].message.tool_calls
    ))

    if response.choices[0].message.tool_calls is not None and len(response.choices[0].message.tool_calls) > 0:
        for tool_call in response.choices[0].message.tool_calls:
            function_args = json.loads(tool_call.function.arguments.replace("'", '"'))
            print(f"Calling function `{tool_call.function.name}` with arguments {function_args}")
            callable_func = globals()[tool_call.function.name]
            function_response = callable_func(**function_args)
            print(f"Function response = {function_response}")

            messages.append(ToolMessage(
                content=function_response,
                tool_call_id=tool_call.id
            ))

        response = aoai_client.chat.completions.create(
            model=aoai_deployment_name, 
            messages=messages,
            tools=tools
        )

print(f"Model response = {response.choices[0].message.content}")

Calling function `get_weather` with arguments {'city': 'Korea'}
Function response = Unavailable
Calling function `get_temperature` with arguments {'city': 'Korea'}
Function response = Unavailable
Model response = It seems that the weather and temperature information for "Korea" is unavailable at the moment. If you need information for a specific city in Korea, please provide the name of that city.


# 4. Set up AIInferenceInstrumentor to test AI Inference tracing
- https://pypi.org/project/azure-ai-inference/
- https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-local-sdk?tabs=python 

In [40]:
from azure.ai.inference.tracing import AIInferenceInstrumentor 

os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = 'true'

# Instrument AI Inference API 
AIInferenceInstrumentor().instrument() 

## Trace function calling with Azure OpenAI SDK

In [41]:
ai_inference_client = ChatCompletionsClient(endpoint=azure_model_inference_endpoint, credential=AzureKeyCredential(aoai_api_key))

In [42]:
question = "Tell me the history of Microsoft"

user_message = f"""
Question: {question}
"""

# Simple API Call
response = ai_inference_client.complete(
    model=aoai_deployment_name,
    messages=[
        {"role": "user", "content": user_message},
    ],
  
)
print(response.choices[0].message.content)

Microsoft Corporation, a multinational technology company, was founded on April 4, 1975, by Bill Gates and Paul Allen in Albuquerque, New Mexico. The company initially focused on creating software for the Altair 8800, an early personal computer. 

### 1970s: Formation and Early Products
- **1975:** Bill Gates and Paul Allen write a version of the BASIC programming language for the Altair 8800, marking Microsoft’s first product.
- **1976:** The company is officially incorporated as "Microsoft," a blend of "microcomputer" and "software."
- **1979:** Microsoft relocates to Bellevue, Washington, to accommodate growth.

### 1980s: Rise to Prominence
- **1980:** Microsoft enters into a contract with IBM to provide an operating system (OS) for their first personal computer (PC). They purchase an OS called QDOS (Quick and Dirty Operating System) and adapt it into MS-DOS.
- **1981:** Microsoft becomes a leading PC software vendor.
- **1985:** Microsoft launches Windows 1.0, a graphical operatin

In [43]:
response = aoai_client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {
            "role": "user",
            "content": "How many feet are in a mile?",
        },
    ],
)

print(response.choices[0].message.content)

There are 5,280 feet in a mile.


In [44]:
messages = [
    SystemMessage("You are a helpful assistant."),
    UserMessage("What is the weather and temperature in Seattle?"),
]

response = ai_inference_client.complete(messages=messages, tools=[weather_description, temperature_in_city], model=aoai_deployment_name)

if response.choices[0].finish_reason == CompletionsFinishReason.TOOL_CALLS:
    # Append the previous model response to the chat history
    messages.append(AssistantMessage(tool_calls=response.choices[0].message.tool_calls))
    # The tool should be of type function call.
    if response.choices[0].message.tool_calls is not None and len(response.choices[0].message.tool_calls) > 0:
        for tool_call in response.choices[0].message.tool_calls:
            if type(tool_call) is ChatCompletionsToolCall:
                function_args = json.loads(tool_call.function.arguments.replace("'", '"'))
                print(f"Calling function `{tool_call.function.name}` with arguments {function_args}")
                callable_func = globals()[tool_call.function.name]
                function_response = callable_func(**function_args)
                print(f"Function response = {function_response}")
                # Provide the tool response to the model, by appending it to the chat history
                messages.append(ToolMessage(function_response, tool_call_id=tool_call.id))
                # With the additional tools information on hand, get another response from the model
        response = ai_inference_client.complete(messages=messages, tools=[weather_description, temperature_in_city], model=aoai_deployment_name)

print(f"Model response = {response.choices[0].message.content}")

Calling function `get_weather` with arguments {'city': 'Seattle'}
Function response = Nice weather
Calling function `get_temperature` with arguments {'city': 'Seattle'}
Function response = 75
Model response = The weather in Seattle is nice, and the current temperature is 75°F.


# 5. Use tracing to view performance and optimize your application

![Tracing in AI Foundry](tracing_in_ai_foundry.png)


![Tracing in AI Foundry](tracing_in_ai_foundry2.png)

# 6. uninstrument the OpenAIInstrumentor and AIInferenceInstrumentor
> 📍 ***important*** <br>
> The OpenAIInstrumentor and AIInferenceInstrumentor are not uninstrumented automatically when the program ends. You must call the uninstrument method to stop tracing.
> This is important to avoid memory leaks and ensure that the tracing data is sent to the Azure Monitor.

In [45]:
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

# Instrument Azure Open AI API 
OpenAIInstrumentor().uninstrument()

In [46]:
from azure.ai.inference.tracing import AIInferenceInstrumentor 

AIInferenceInstrumentor().uninstrument()
# [END uninstrument_inferencing]